In [1]:
%pylab
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
import pandas as pd
import seaborn as sns
path =  (os.path.abspath(os.path.join(r'.', os.pardir)))
import sys
sys.path.append(path)
from images_segmentation import otsu as ot
from images_segmentation import dicescore as dsc
from PIL import Image
import os
import os.path

Using matplotlib backend: <object object at 0x0000020F7A6526F0>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Set ground truth images from N2DH-GOWT1 dataset and N2DL-HeLa dataset to 0 and 1

def binarize(x):
    img = x.copy()

    for o in np.ndindex(img.shape):
        if img[o] > 0: 
           img[o] = 1
        else:
            img[o] = 0
    
    return img

## NIH3T3 Dataset

In [4]:
# Load raw images of NIH3T3-dataset

img_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/img")
for f in os.listdir(path):
    img_NIH3T3.append(imread(os.path.join(path , f)))

# Load ground truth images of NIH3T3-dataset
 
gt_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/gt")
for f in os.listdir(path):
    gt_NIH3T3.append(imread(os.path.join(path , f)))

In [5]:
# Make a copy of all images of each dataset / use as "no preprocessed images"
copy_img_NIH3T3 = img_NIH3T3.copy()

#### Median filter

In [6]:
# Apply median filter to all images of each dataset
# mit kleinerern Filtern läuft es schneller durch
# filter größen die man möchte
filter = [3, 4, 5, 10, 15, 20]
median_img_NIH3T3 = []
for j in range (0, len(filter)):
    
    for i in range(0, len(copy_img_NIH3T3)):
        median_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[j]))
    print("filtersize done")    

filtersize done
filtersize done
filtersize done
filtersize done
filtersize done
filtersize done


#### gaussian filter

In [ ]:
# Apply gaussian filter to all images of each dataset
#filter größen die mach möchte 
filter_g = [3, 4, 5, 10, 15, 20]
gauss_img_NIH3T3 = []
for j in range (0, len(filter_g)):

    for i in range(0,len(copy_img_NIH3T3)):
        gauss_img_NIH3T3.append(scipy.ndimage.gaussian_filter(copy_img_NIH3T3[i] , size= filter_g[j]))
    print("filtersize done")

## Otsu thresholding

In [8]:
# Apply otsu thresholding to all images of each dataset - median filter
otsu_median_img_NIH3T3 = []
for i in range(0,len(median_img_NIH3T3)):
    otsu_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [i] , 256))




In [ ]:
# Apply otsu thresholding to all images of each dataset - gaussian filter
otsu_gauss_img_NIH3T3 = []
for i in range(0,len(gauss_img_NIH3T3)):
    otsu_gauss_img_NIH3T3.append(ot.otsu_thresholding(gauss_img_NIH3T3[i] , 256))


## Dice score

In [21]:
# Dice score - NIH3T3-dataset
## median filter

gt_NIH3T3_6 = [gt_NIH3T3, gt_NIH3T3, gt_NIH3T3, gt_NIH3T3, gt_NIH3T3, gt_NIH3T3]
dice_otsu_median_img_NIH3T3 = []
for j in range( 0, len(otsu_median_img_NIH3T3)):
    dice_otsu_median_img_NIH3T3.append(dsc.dice(otsu_median_img_NIH3T3[j] , gt_NIH3T3_6[j]))
print(dice_otsu_median_img_NIH3T3)

TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [ ]:
# Dice score - N2DH-GOWT1-dataset
## gaussian filter

dice_otsu_gauss_img_N2DH_GOWT1 = []
for j in range(0,len(otsu_gauss_img_N2DH_GOWT1)):
    dice_otsu_gauss_img_N2DH_GOWT1.append(dsc.dice(otsu_gauss_img_N2DH_GOWT1[j] , binary_gt_N2DH_GOWT1[j]))

print(dice_otsu_gauss_img_N2DH_GOWT1)

NameError: name 'otsu_gauss_img_N2DH_GOWT1' is not defined